# Machine Learning em Química

## Caso de estudo 1: Classificação

O primeiro caso de estudo tem por objectivo criar um modelo classificador para moléculas com potencial odor a almíscar.

In [ ]:
# recolha dos dados
!wget https://tinyurl.com/fvkedcjd
!mv fvkedcjd exemplo_1_dados.xlsx
# carregar dados usando o pandas
import pandas as pd

dados = pd.read_excel('exemplo_1_dados.xlsx')

In [ ]:
# observar os dados
dados

In [ ]:
# criar um sumário dos dados
dados.describe()

In [ ]:
# divisão entre dados de treino e de teste
from sklearn import model_selection

treino, teste = model_selection.train_test_split(dados, train_size=0.6, stratify=dados['Is_musk'], random_state=42)

target='Is_musk'
features=[f"f{x:d}" for x in range(1,167)]
print(features)

### Primeira abordagem: PCA

O PCA aparece como método de redução de dimensionalidade que permite identificar agrupamentos nos dados. Caso a localização ao longo de um dado PC permita estabelelecer uma divisão entre casos positivos e negativos, podemos usar esse PC como parte de um modelo classificador.

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components=10)
#treinamos o modelo PCA
pca.fit_transform(treino[features])
print(pca.explained_variance_ratio_)

In [ ]:
scores_treino = pca.transform(treino[features])
scores_teste = pca.transform(teste[features])

import matplotlib.pyplot as plt

fg, ax = plt.subplots()
s1 = ax.scatter(scores_treino[:,0], scores_treino[:,1], c=treino[target], cmap='bwr', label='Treino', marker='o')
s2 = ax.scatter(scores_teste[:,0], scores_teste[:,1], c=teste[target], cmap='PuOr', label='Treino', marker='v')
ax.set_xlabel("PC1 (35 %)")
ax.set_ylabel("PC2 (20 %)")

ax.legend()
plt.show()

### Segunda Abordagem: Regressão Logística
Apesar do seu nome, a regressão logística é um método de classificação supervisionada.

In [ ]:
from sklearn import linear_model

clf = linear_model.LogisticRegression(max_iter=5000)
clf.fit(treino[features], treino[target])

# calculamos o score (excatidão) para o treino e para o teste
s_treino = clf.score(treino[features], treino[target])
s_teste = clf.score(teste[features], teste[target])

print(f"Score no treino: {s_treino:6.4f}")
print(f"Score no teste : {s_teste:6.4f}")

Para avaliar o sistema, usamos uma ou mais métricas.

In [ ]:
from sklearn import metrics
acc_treino = metrics.accuracy_score(treino[target],clf.predict(treino[features]))
acc_teste = metrics.accuracy_score(teste[target],clf.predict(teste[features]))
print(f"Exatidão no treino: {acc_treino:0.4f}")
print(f"Exatidão no teste: {acc_teste:0.4f}")

In [ ]:
acc_treino = metrics.precision_score(treino[target],clf.predict(treino[features]))
acc_teste = metrics.precision_score(teste[target],clf.predict(teste[features]))
print(f"Precisão no treino: {acc_treino:0.4f}")
print(f"Precisão no teste: {acc_teste:0.4f}")

In [ ]:
acc_treino = metrics.recall_score(treino[target],clf.predict(treino[features]))
acc_teste = metrics.recall_score(teste[target],clf.predict(teste[features]))
print(f"Recall no treino: {acc_treino:0.4f}")
print(f"Recall no teste: {acc_teste:0.4f}")

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(8,3))
axs[0].set_title("ROC AUC com os dados de treino")
metrics.RocCurveDisplay.from_estimator(clf,treino[features],treino[target],ax=axs[0])
axs[1].set_title("ROC AUC com os dados de teste")
metrics.RocCurveDisplay.from_estimator(clf,teste[features],teste[target],ax=axs[1])
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].set_title("Matriz de confusão com os dados de treino")
metrics.ConfusionMatrixDisplay.from_estimator(clf,treino[features],treino[target],normalize='true',ax=axs[0], cmap="summer")
axs[1].set_title("Matriz de confusão com os dados de teste")
metrics.ConfusionMatrixDisplay.from_estimator(clf,teste[features],teste[target],normalize='true',ax=axs[1], cmap="summer")
plt.show()

## Caso de estudo 2: Classificação Multicategórica


In [ ]:
# recolha dos dados
!wget https://tinyurl.com/4yzbe8h9
!mv 4yzbe8h9 exemplo_2_dados.xlsx
# carregar dados usando o pandas
import pandas as pd

dados = pd.read_excel('exemplo_2_dados.xlsx')

In [ ]:
dados.describe()

In [ ]:
target='Type'
features=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe']

treino, teste = model_selection.train_test_split(dados, train_size=0.6, stratify=dados[target], random_state=42)

### Classificação por Árvore de Decisão

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(treino[features], treino[target])

In [ ]:
# Nem todas as métricas são diretamente aplicávais a casos não binários..
acc_treino = metrics.accuracy_score(treino[target],clf.predict(treino[features]))
acc_teste = metrics.accuracy_score(teste[target],clf.predict(teste[features]))
print(f"Exatidão no treino: {acc_treino:0.4f}")
print(f"Exatidão no teste: {acc_teste:0.4f}")



fig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].set_title("Matriz de confusão com os dados de treino")
metrics.ConfusionMatrixDisplay.from_estimator(clf,treino[features],treino[target],normalize='true',ax=axs[0], cmap="summer")
axs[1].set_title("Matriz de confusão com os dados de teste")
metrics.ConfusionMatrixDisplay.from_estimator(clf,teste[features],teste[target],normalize='true',ax=axs[1], cmap="summer")
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
tree.plot_tree(clf, max_depth=3,feature_names=features, class_names=['1','2','3','5','6','7'], label='root', filled=True, ax=ax, fontsize=9, rounded=True)
plt.show()

In [ ]:
from sklearn import ensemble

clf = ensemble.RandomForestClassifier(random_state=42)
clf.fit(treino[features], treino[target])

acc_treino = metrics.accuracy_score(treino[target],clf.predict(treino[features]))
acc_teste = metrics.accuracy_score(teste[target],clf.predict(teste[features]))
print(f"Exatidão no treino: {acc_treino:0.4f}")
print(f"Exatidão no teste: {acc_teste:0.4f}")

fig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].set_title("Matriz de confusão com os dados de treino")
metrics.ConfusionMatrixDisplay.from_estimator(clf,treino[features],treino[target],normalize='true',ax=axs[0], cmap="summer")
axs[1].set_title("Matriz de confusão com os dados de teste")
metrics.ConfusionMatrixDisplay.from_estimator(clf,teste[features],teste[target],normalize='true',ax=axs[1], cmap="summer")
plt.show()

In [ ]:
# optimizacao dos hiper-parametros

clf_base = ensemble.RandomForestClassifier(random_state=42)

# criamos um dicionario com os valores a dar a cada hiper-parametro
params = {'n_estimators': [10, 25, 50, 100, 150, 200],
          'max_samples': [0.10, 0.25, 0.5, 0.75, 1.0],
          'max_features': [0.10, 0.25, 0.5, 0.75, 1.0]}

clf = model_selection.GridSearchCV(clf_base, params, refit=True)

clf.fit(treino[features],treino[target])

In [ ]:
import seaborn as sns
cv_data = pd.DataFrame(clf.cv_results_)
sns.boxplot(x='param_n_estimators', y='mean_test_score', data=cv_data)


In [ ]:
sns.boxplot(x='param_max_features', y='mean_test_score', data=cv_data)

In [ ]:
sns.boxplot(x='param_max_samples', y='mean_test_score', data=cv_data)

In [ ]:
print(clf.best_params_)

In [ ]:
acc_treino = metrics.accuracy_score(treino[target],clf.predict(treino[features]))
acc_teste = metrics.accuracy_score(teste[target],clf.predict(teste[features]))
print(f"Exatidão no treino: {acc_treino:0.4f}")
print(f"Exatidão no teste: {acc_teste:0.4f}")

fig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].set_title("Matriz de confusão com os dados de treino")
metrics.ConfusionMatrixDisplay.from_estimator(clf,treino[features],treino[target],normalize='true',ax=axs[0], cmap="summer")
axs[1].set_title("Matriz de confusão com os dados de teste")
metrics.ConfusionMatrixDisplay.from_estimator(clf,teste[features],teste[target],normalize='true',ax=axs[1], cmap="summer")
plt.show()